In [2]:
%cd cmu-mosei-experiments/CMU-MultimodalSDK-Tutorials/

/cephfs/volumes/hpc_data_usr/k24083007/2070c87e-fe07-4f03-a6c4-cae0de8ce617/cmu-mosei-experiments/CMU-MultimodalSDK-Tutorials


In [3]:
import os
from mmsdk import mmdatasdk as md
from constants.paths import DATA_PATH

In [14]:
import numpy as np

In [4]:
# Recipes for both visual features
visual_comparison_recipe = {
    'VisualFacet42': os.path.join(DATA_PATH, 'CMU_MOSEI_VisualFacet42.csd'),
    'OpenFace2': os.path.join(DATA_PATH, 'CMU_MOSEI_OpenFace2.csd')
}

# Load both visual feature sets
try:
    visual_dataset = md.mmdataset(visual_comparison_recipe)
    print("Both visual feature sets loaded successfully!")
    print("Available features:", list(visual_dataset.keys()))
except Exception as e:
    print(f"Error loading visual features: {e}")

[2025-06-30 14:40:27.421] | Success | Computational sequence read from file ./data/CMU_MOSEI_VisualFacet42.csd ...
[2025-06-30 14:40:27.494] | Status  | Checking the integrity of the <FACET 4.2> computational sequence ...
[2025-06-30 14:40:27.494] | Status  | Checking the format of the data in <FACET 4.2> computational sequence ...


[2025-06-30 14:40:28.444] | Success | <FACET 4.2> computational sequence data in correct format.
[2025-06-30 14:40:28.444] | Status  | Checking the format of the metadata in <FACET 4.2> computational sequence ...
[2025-06-30 14:40:28.444] | Warning | <FACET 4.2> computational sequence does not have all the required metadata ... continuing 
[2025-06-30 14:40:28.445] | Success | Computational sequence read from file ./data/CMU_MOSEI_OpenFace2.csd ...
[2025-06-30 14:40:28.515] | Status  | Checking the integrity of the <OpenFace_2> computational sequence ...
[2025-06-30 14:40:28.515] | Status  | Checking the format of the data in <OpenFace_2> computational sequence ...


[2025-06-30 14:40:29.437] | Success | <OpenFace_2> computational sequence data in correct format.
[2025-06-30 14:40:29.437] | Status  | Checking the format of the metadata in <OpenFace_2> computational sequence ...
[2025-06-30 14:40:29.437] | Warning | <OpenFace_2> computational sequence does not have all the required metadata ... continuing 
[2025-06-30 14:40:29.437] | Success | Dataset initialized successfully ... 
Both visual feature sets loaded successfully!
Available features: ['VisualFacet42', 'OpenFace2']


In [13]:
# Dimensionality analysis
print("=" * 60)
print("Dimensionality Analysis")
print("=" * 60)

print("Available fields in visual dataset:", list(visual_dataset.computational_sequences.keys()))
facet42_field = list(visual_dataset.computational_sequences.keys())[0]
openface2_field = list(visual_dataset.computational_sequences.keys())[1]

# Get the total number of segments
total_segments = len(visual_dataset[facet42_field].keys())
print(f"Total segments in dataset: {total_segments}")

# Sample segments are the first 10 visual segments
sample_segments = list(visual_dataset[facet42_field].keys())[:10]
facet42_dims = []
openface2_dims = []

# Taking one segment to analyze everything henceforth
sample_segment = sample_segments[0]
try:
    facet42_features = visual_dataset['VisualFacet42'][sample_segment]['features']
    openface2_features = visual_dataset['OpenFace2'][sample_segment]['features']
    
    facet42_dims.append(facet42_features.shape[1])  # Feature dimension
    openface2_dims.append(openface2_features.shape[1])
    
    print(f"\nSegment {sample_segment}:")
    print(f"  VisualFacet42: {facet42_shape} (Time steps x Features)")
    print(f"  OpenFace2:     {openface2_shape} (Time steps x Features)")
except KeyError:
    print(f"Segment {sample_segment} not found in dataset.")

print(f"\nFeature Dimensions:")
print(f"VisualFacet42: {facet42_dims[0]} features")
print(f"OpenFace2:     {openface2_dims[0]} features")

Dimensionality Analysis
Available fields in visual dataset: ['VisualFacet42', 'OpenFace2']
Total segments in dataset: 3837

Segment --qXJuDtHPw:
  VisualFacet42: (1715, 35) (Time steps x Features)
  OpenFace2:     (1714, 713) (Time steps x Features)

Feature Dimensions:
VisualFacet42: 35 features
OpenFace2:     713 features


CMU SDK has (features, intervals) defined as the arrays

Features array shape is (time_steps, 35) for VisualFacet42
Intervals array shape is (time_steps, 2)

In [17]:
# Stats analysis
print("=" * 60)
print("Stats Analysis")
print("=" * 60)

print(f"VisualFacet42 Stats:")
print(f"  Mean: {np.mean(facet42_features):.4f}")
print(f"  Std:  {np.std(facet42_features):.4f}")
print(f"  Range: [{np.min(facet42_features):.4f}, {np.max(facet42_features):.4f}]")
print(f"  Sparsity (% zeros): {np.mean(facet42_features == 0) * 100:.2f}%")

print(f"\nOpenFace2 Stats:")
print(f"  Mean: {np.mean(openface2_features):.4f}")
print(f"  Std:  {np.std(openface2_features):.4f}")
print(f"  Range: [{np.min(openface2_features):.4f}, {np.max(openface2_features):.4f}]")
print(f"  Sparsity (% zeros): {np.mean(openface2_features == 0) * 100:.2f}%")

Stats Analysis
VisualFacet42 Stats:
  Mean: -0.8856
  Std:  2.0401
  Range: [-18.2228, 16.4741]
  Sparsity (% zeros): 0.00%

OpenFace2 Stats:
  Mean: 166.3502
  Std:  451.4024
  Range: [-336.6000, 1353.4000]
  Sparsity (% zeros): 0.00%


OpenFace2 -> larger range, extreme outliers -> necessitating aggressive normalization

In [19]:
# Temporal Dynamics analysis
print("=" * 60)
print("Temporal Dynamics Analysis")
print("=" * 60)

# Analysing temporal smoothness
def temporal_variation(features):
    """Calculate frame-to-frame variation to measure temporal smoothness"""
    if len(features) < 2:
        return 0
    return np.mean(np.sum(np.abs(np.diff(features, axis=0)), axis=1))

facet42_temporal_var = temporal_variation(facet42_features)
openface2_temporal_var = temporal_variation(openface2_features)

print(f"Temporal Variation:")
print(f"  VisualFacet42: {facet42_temporal_var:.4f}")
print(f"  OpenFace2:     {openface2_temporal_var:.4f}")

# Feature variance analysis across time
facet42_feature_var = np.var(facet42_features, axis=0)
openface2_feature_var = np.var(openface2_features, axis=0)

print(f"\nFeature Variance Statistics:")
print(f"VisualFacet42:")
print(f"  High variance features (>1.0): {np.sum(facet42_feature_var > 1.0)}")
print(f"  Low variance features (<0.1): {np.sum(facet42_feature_var < 0.1)}")
print(f"  Mean variance: {np.mean(facet42_feature_var):.4f}")

print(f"OpenFace2:")
print(f"  High variance features (>1.0): {np.sum(openface2_feature_var > 1.0)}")
print(f"  Low variance features (<0.1): {np.sum(openface2_feature_var < 0.1)}")
print(f"  Mean variance: {np.mean(openface2_feature_var):.4f}")

Temporal Dynamics Analysis
Temporal Variation:
  VisualFacet42: 5.8701
  OpenFace2:     942.8901

Feature Variance Statistics:
VisualFacet42:
  High variance features (>1.0): 6
  Low variance features (<0.1): 4
  Mean variance: 1.9168
OpenFace2:
  High variance features (>1.0): 635
  Low variance features (<0.1): 51
  Mean variance: 314.0168


Lower temporal variation means smoother frame by frame changes

VisualFacet42 -> Low variance, 
OpenFace -> High variance -> noise and over-sensitivity in its high-dimensional outputs

Choose VisualFacet42 because:
- semantically aligned emotion scores, 
- compact yet rich embeddings, and 
- smooth temporal behavior

- High-dimensional OpenFace2 (713 dims) demands PCA, autoencoders, or heavy regularization to avoid overfitting, especially on CMU-MOSEI’s ~4,000 segments

- VisualFacet42 was designed for affective analytics, encapsulating key AUs and continuous metrics (valence, engagement) that correlate strongly with emotional states

- VisualFacet42’s per‐feature stability and built-in confidence scores (quality flags) enable seamless aleatoric uncertainty modeling via evidential frameworks, unlike OpenFace’s noisy raw landmarks

